In [6]:
!pip install chromadb

In [7]:
!pip install langchain langchain-community

In [8]:
!pip install -qU langchain-chroma

In [32]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama

In [10]:
import os
from getpass import getpass
HF_token=getpass()

··········


In [11]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=HF_token

In [12]:
URL= "https://en.wikipedia.org/wiki/Palakkad_district"

In [13]:
data=WebBaseLoader(URL)

In [14]:
con=data.load()

In [15]:
print(con)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Palakkad_district', 'title': 'Palakkad district - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nPalakkad district - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(T

##Text splitter

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350,chunk_overlap=0)

In [17]:
chunking = text_splitter.split_documents(con)
len(chunking)

208

In [18]:
chunking[50]

Document(metadata={'source': 'https://en.wikipedia.org/wiki/Palakkad_district', 'title': 'Palakkad district - Wikipedia', 'language': 'en'}, page_content='10\nChittur\nK. Krishnankutty\nJD(S)\n\xa0\xa0LDF\n\n\n11\nNenmara\nK. Babu\nCPI(M)\n\xa0\xa0LDF\n\n\n12\nAlathur\nK. D. Prasenan\nCPI(M)\n\xa0\xa0LDF\n\nMembers of Lok Sabha[edit]\nThere are two Lok Sabha seats in Palakkad district.\n\n\n\nSl. No\n\nConstituency\n\nMP\n\nParty\n\nAlliance\n\n\n1\nPalakkad\nV.K Sreekandan\nINC\n\xa0\xa0UDF\n\n\n2\nAlathur\nK. Radhakrishnan\nCPI(M)\n\xa0\xa0LDF')

In [19]:
chunking[20]

Document(metadata={'source': 'https://en.wikipedia.org/wiki/Palakkad_district', 'title': 'Palakkad district - Wikipedia', 'language': 'en'}, page_content='largest being the Malampuzha dam. The largest in volume capacity is the Parambikulam Dam[9] Bhavani River, which is a tributary of Kaveri River, also flows through the district. Kadalundi River has its origin in Silent Valley National Park. The Chalakudy River also flows through district.')

In [20]:
chunking

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Palakkad_district', 'title': 'Palakkad district - Wikipedia', 'language': 'en'}, page_content='Palakkad district - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Palakkad_district', 'title': 'Palakkad district - Wikipedia', 'language': 'en'}, page_content='Search\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn mo

#EMbedding Model

In [21]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_token,model_name="BAAI/bge-base-en-v1.5")

In [22]:
vectorstore = Chroma.from_documents(chunking,embeddings)

In [23]:
model=HuggingFaceHub(repo_id="deepseek-ai/deepseek-coder-1.3b-base",model_kwargs={"temperature":0.5,"max_new_tokens":256,"max_length":64})

<ipython-input-23-04b7d7f82cb9>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model=HuggingFaceHub(repo_id="deepseek-ai/deepseek-coder-1.3b-base",model_kwargs={"temperature":0.5,"max_new_tokens":256,"max_length":64})


In [24]:
retriever=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":5})

In [25]:
query = "what is the palakkad district population"

In [26]:
docs = retriever.get_relevant_documents(query)

<ipython-input-26-aef05c160744>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [27]:
prompt = f"""
<|system|>
You are a helpful assistant.
</s>
<|user|>
{query}
</s>
<|assistant/>
"""

In [28]:
docs

[Document(id='d7ac9ed3-a8de-421b-959c-44823e8637e9', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/Palakkad_district', 'title': 'Palakkad district - Wikipedia'}, page_content='In 2011, Palakkad had population of 2,809,934 of which male and female were 1,359,478 and 1,450,456 respectively. In 2001 census, Palakkad had a population of 2,617,482 of which males were 1,266,985 and remaining 1,350,497 were females. Palakkad District population constituted 8.41 percent of total Kerala population in the census of 2011. In 2001'),
 Document(id='627ab76d-032a-4306-9977-74d4a53b875b', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/Palakkad_district', 'title': 'Palakkad district - Wikipedia'}, page_content='According to the 2018 Statistics Report, Palakkad district has a population of 2,952,254,[2] roughly equal to the nation of Armenia[41] or the state Utah in the U.S.[42] so it a ranking of 138th in India (out of a total of 640).[7] The district has a

In [29]:
q=RetrievalQA.from_chain_type(llm=model,retriever=retriever,chain_type="stuff")

In [30]:
response = q(prompt)

<ipython-input-30-871e1f9477c0>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = q(prompt)


In [31]:
print(response['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

According to the 2018 Statistics Report, Palakkad district has a population of 2,952,254,[2] roughly equal to the nation of Armenia[41] or the state Utah in the U.S.[42] so it a ranking of 138th in India (out of a total of 640).[7] The district has a population density of 659 inhabitants per square kilometre (1,710/sq mi).[7] Its population growth

Palakkad District.

Languages[edit]









Languages of Palakkad district (2011)[4]

  Malayalam (93.71%)  Tamil (4.82%)  Others (1.47%)

Subdistrict

Area(in km2)

Population(2011)

Villages


Pattambi
369
467,722
18


Ottapalam
477
462,970
24


Mannarkkad
1,209
384,393
25


Palakkad
713
612,116
30


Chittur
1,136
437,738
30


Alathur
571
444,995
30


Sources: 2011 Census of India,[34] Official website of Palakkad district[35]

Categories: Villages in Palakkad districtPalakkad 